In [1]:
import numpy as np
import pandas as pd

import wandb
from config import Config

wandb.init(project="NeuroWood2022-name", entity="nuclear_foxes_team")

wandb: Currently logged in as: nuclear_foxes_team (use `wandb login --relogin` to force relogin)


In [2]:
import source

In [3]:
import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
from torch.optim import Adam, SGD
from torch.utils.data import DataLoader

import torchvision


torch.manual_seed(7575)
np.random.seed(7575)
torch.cuda.empty_cache()

## Create Dataset

In [4]:
output_dataframe = pd.read_csv('sample_submission.csv')
output_dataframe['id'] = output_dataframe['id'].astype(str)
output_dataframe.head()

,id,class
0,1,NaN
1,2,NaN
2,3,NaN
3,4,NaN
4,5,NaN


In [5]:
train_dataset = source.WoodDataset()
test_dataset = source.WoodDataset(create_train=False)

train_dataloader = DataLoader(train_dataset, batch_size=Config.BATCH_SIZE, pin_memory=True, shuffle=True)
test_dataloader = DataLoader(test_dataset, pin_memory=True, batch_size=Config.BATCH_SIZE)

## Model

In [6]:
main_model = torchvision.models.vgg11_bn(pretrained=True)

class ModelFinal(nn.Module):
    def __init__(self):
        super(ModelFinal, self).__init__()
        self.main_model = main_model
        fc = nn.Sequential(
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 3),
            nn.Softmax(dim=1)
        )
        self.main_model.fc = fc
    def forward(self, x):
        return self.main_model.forward(x)

In [7]:
model = ModelFinal()

lr = 8e-4
parameters = {
    'optimizer' : SGD(model.parameters(), lr=lr),
    'criterion' : CrossEntropyLoss(),
    'n_epochs' : 15,
    'device' : Config.DEVICE
}

wandb_dict = {
    "Learning_rate": lr,
    "Epochs": parameters['n_epochs'],
    "Batch_size": Config.BATCH_SIZE,
    "Network": "VGG11BN + SGD",
    "Full train dataset" : True
}

wandb.config.update(wandb_dict)

In [8]:
source.train(model, train_dataloader, **parameters)
path = source.save_model(model, wandb_dict['Network'])
wandb_dict["Path"] = path
wandb.config.update(wandb_dict)

100%|██████████| 145/145 [02:18<00:00,  1.05it/s]


Epoch 1/15 is finished, train loss is 0.3794178524857451.


100%|██████████| 145/145 [02:14<00:00,  1.08it/s]


Epoch 2/15 is finished, train loss is 0.0953379252323367.


100%|██████████| 145/145 [02:15<00:00,  1.07it/s]


...

Epoch 13/15 is finished, train loss is 0.007374809839103395.


100%|██████████| 145/145 [02:17<00:00,  1.06it/s]


Epoch 14/15 is finished, train loss is 0.006229986167705521.


100%|██████████| 145/145 [02:16<00:00,  1.06it/s]


Epoch 15/15 is finished, train loss is 0.005459510067991906.


In [9]:
predicts = source.predict(model, test_dataloader)

100%|██████████| 63/63 [00:53<00:00,  1.18it/s]


In [10]:
output_dataframe['class'] = predicts
output_dataframe['class'] = output_dataframe['class'].astype(int)

In [11]:
def transform(x):
    if (x == 0):
        return x
    elif (x == 1):
        return x
    else:
        return 3

In [12]:
output_dataframe['class'] = output_dataframe['class'].apply(transform)
output_dataframe.to_csv('prediction.csv', index=False)